In [1]:
import numpy as np
from typing import List, Tuple, Dict, Any

#  Интервальная реализация VIKOR
def interval_vikor(
    matrix: np.ndarray,          
    weights: List[float],        
    types: List[int],            
    v: float = 0.5
) -> Dict[str, Any]:
    # Возвращает интервальные S, R, Q и ранжирование.

    m, n = matrix.shape[0], matrix.shape[1]
    L = matrix[:, :, 0]
    U = matrix[:, :, 1]

    # 1) интервальные идеал/надир для каждого критерия
    f_star = np.zeros((n, 2))   # [low, high]
    f_minus = np.zeros((n, 2))  # [low, high]

    for j in range(n):
        if types[j] == 1:  # benefit
            f_star[j, 0] = np.max(L[:, j])   # best lower
            f_star[j, 1] = np.max(U[:, j])   # best upper
            f_minus[j, 0] = np.min(L[:, j])  # worst lower
            f_minus[j, 1] = np.min(U[:, j])  # worst upper
        else:  # cost
            f_star[j, 0] = np.min(L[:, j])
            f_star[j, 1] = np.min(U[:, j])
            f_minus[j, 0] = np.max(L[:, j])
            f_minus[j, 1] = np.max(U[:, j])

    print("\n=== Шаг 1. Интервальные идеал f* и надир f- ===")
    for j in range(n):
        print(
            f"Критерий {j+1}: "
            f"f* = [{f_star[j,0]:.3f}, {f_star[j,1]:.3f}], "
            f"f- = [{f_minus[j,0]:.3f}, {f_minus[j,1]:.3f}]"
        )


    # 2) вычисляем интервальные нормализованные разрывы D_ij = [D_low, D_high]
    # D_low = (f*_low - overline f_ij) / (f*_low - f-_high)
    # D_high = (f*_high - underline f_ij) / (f*_high - f-_low)
    # будет применяться для каждого критерия и альтернативы.
    D_low = np.zeros((m, n))
    D_high = np.zeros((m, n))

    for i in range(m):
        for j in range(n):
            fstar_low = f_star[j, 0]
            fstar_high = f_star[j, 1]
            fminus_low = f_minus[j, 0]
            fminus_high = f_minus[j, 1]

            over_ij = U[i, j]
            under_ij = L[i, j]

            denom_low = (fstar_low - fminus_high)
            denom_high = (fstar_high - fminus_low)

            if denom_low == 0:
                d_low = 0.0
            else:
                d_low = (fstar_low - over_ij) / denom_low

            if denom_high == 0:
                d_high = 0.0
            else:
                d_high = (fstar_high - under_ij) / denom_high

            D_low[i, j] = np.clip(d_low, 0.0, 1.0)
            D_high[i, j] = np.clip(d_high, 0.0, 1.0)


    print("\n=== Шаг 2. Интервальные нормализованные разрывы D_ij ===")
    for i in range(m):
        print(f"Альтернатива A{i+1}:")
        for j in range(n):
            print(
                f"  Критерий {j+1}: "
                f"D = [{D_low[i,j]:.3f}, {D_high[i,j]:.3f}]"
            )
        
    # 3) агрегируем по весам: интервальные S_i и R_i
    weights = np.array(weights)
    S = np.zeros((m, 2))  # [S_low, S_high]
    R = np.zeros((m, 2))  # [R_low, R_high]

    for i in range(m):
        # суммирование по низким и по высоким границам
        S[i, 0] = np.sum(weights * D_low[i, :])
        S[i, 1] = np.sum(weights * D_high[i, :])
        # R = max_j w_j * D_ij (по компонентам)
        R[i, 0] = np.max(weights * D_low[i, :]) if n > 0 else 0.0
        R[i, 1] = np.max(weights * D_high[i, :]) if n > 0 else 0.0


    print("\n=== Шаг 3. Интервальные метрики S_i и R_i ===")
    for i in range(m):
        print(
            f"A{i+1}: "
            f"S = [{S[i,0]:.3f}, {S[i,1]:.3f}], "
            f"R = [{R[i,0]:.3f}, {R[i,1]:.3f}]"
        )
    
    # 4) вычисляем S*, S-, R*, R- для комбинаций границ
    S_star_low = np.min(S[:, 0])
    S_star_high = np.min(S[:, 1])
    S_minus_low = np.max(S[:, 0])
    S_minus_high = np.max(S[:, 1])

    R_star_low = np.min(R[:, 0])
    R_star_high = np.min(R[:, 1])
    R_minus_low = np.max(R[:, 0])
    R_minus_high = np.max(R[:, 1])


    print("\n=== Шаг 4. Интервальные экстремумы ===")
    print(f"S*: [{S_star_low:.3f}, {S_star_high:.3f}]")
    print(f"S-: [{S_minus_low:.3f}, {S_minus_high:.3f}]")
    print(f"R*: [{R_star_low:.3f}, {R_star_high:.3f}]")
    print(f"R-: [{R_minus_low:.3f}, {R_minus_high:.3f}]")

    # 5) вычисляем интервальный Q: левый и правый концы
    Q = np.zeros((m, 2))
    # предохраняемся от деления на ноль
    def safe_div(numer, denom):
        return numer / denom if abs(denom) > 1e-12 else 0.0

    for i in range(m):
        # левый (пессимистичный)
        num_S_left = S[i, 0] - S_star_high
        den_S_left = S_minus_low - S_star_high
        term_S_left = safe_div(num_S_left, den_S_left)

        num_R_left = R[i, 0] - R_star_high
        den_R_left = R_minus_low - R_star_high
        term_R_left = safe_div(num_R_left, den_R_left)

        q_left = v * term_S_left + (1.0 - v) * term_R_left

        # правый (оптимистичный)
        num_S_right = S[i, 1] - S_star_low
        den_S_right = S_minus_high - S_star_low
        term_S_right = safe_div(num_S_right, den_S_right)

        num_R_right = R[i, 1] - R_star_low
        den_R_right = R_minus_high - R_star_low
        term_R_right = safe_div(num_R_right, den_R_right)

        q_right = v * term_S_right + (1.0 - v) * term_R_right

        # защита: Q в [0,1]
        Q[i, 0] = np.clip(q_left, 0.0, 1.0)
        Q[i, 1] = np.clip(q_right, 0.0, 1.0)

    print("\n=== Шаг 5. Интервальная мера компромисса Q_i ===")
    for i in range(m):
        print(
            f"A{i+1}: "
            f"Q = [{Q[i,0]:.3f}, {Q[i,1]:.3f}]"
        )

    # 6) ранжирование (по центрам интервалов Q)
    Q_centers = (Q[:, 0] + Q[:, 1]) / 2.0
    ranking = np.argsort(Q_centers)

    print("\n=== Шаг 6. Центры интервалов и ранжирование ===")
    for idx in ranking:
        print(
            f"A{idx+1}: Q_center = {Q_centers[idx]:.3f}"
        )

    # 7) проверки компромиссного решения
    sorted_centers = Q_centers[ranking]
    DQ = 1.0 / (m - 1) if m > 1 else 1.0

    # C1: Acceptable Advantage
    condition_C1 = (len(sorted_centers) > 1 and (sorted_centers[1] - sorted_centers[0]) >= DQ)

    # C2: Acceptable Stability
    S_centers = (S[:, 0] + S[:, 1]) / 2.0
    R_centers = (R[:, 0] + R[:, 1]) / 2.0
    best_idx = int(ranking[0])
    condition_C2 = (best_idx == int(np.argmin(S_centers)) or best_idx == int(np.argmin(R_centers)))

    print("\n=== Шаг 7. Проверка условий компромиссного решения ===")
    print(f"Условие C1 (Acceptable Advantage) выполнено: {condition_C1}")
    print(f"Условие C2 (Acceptable Stability) выполнено: {condition_C2}")

    # Определяем итоговое "лучшее" решение
    if condition_C1 and condition_C2:
        best_alternative = best_idx
    else:
        # Если условия не выполняются, допускаем топ-2 как компромисс
        best_alternative = [int(ranking[0]), int(ranking[1])] if m > 1 else best_idx

    return {
        'ranking': ranking.tolist(),
        'Q': Q.tolist(),
        'S': S.tolist(),
        'R': R.tolist(),
        'best_alternative': best_alternative,
        'conditions_met': bool(condition_C1 and condition_C2),
        'Q_centers': Q_centers.tolist(),
        'f_star': f_star.tolist(),
        'f_minus': f_minus.tolist(),
        'D_low': D_low.tolist(),
        'D_high': D_high.tolist(),
        'condition_C1': condition_C1,
        'condition_C2': condition_C2
    }


if __name__ == "__main__":
    matrix = np.array([
        [[0.75, 1.24], [2784, 3192]],  # A1
        [[1.83, 2.11], [3671, 3857]],  # A2
        [[4.90, 5.73], [4409, 4681]],  # A3
    ])
    weights = [0.5, 0.5]
    types = [-1, 1]
    res = interval_vikor(matrix, weights, types, v=0.5)


=== Шаг 1. Интервальные идеал f* и надир f- ===
Критерий 1: f* = [0.750, 1.240], f- = [4.900, 5.730]
Критерий 2: f* = [4409.000, 4681.000], f- = [2784.000, 3192.000]

=== Шаг 2. Интервальные нормализованные разрывы D_ij ===
Альтернатива A1:
  Критерий 1: D = [0.098, 0.000]
  Критерий 2: D = [1.000, 1.000]
Альтернатива A2:
  Критерий 1: D = [0.273, 0.161]
  Критерий 2: D = [0.454, 0.532]
Альтернатива A3:
  Критерий 1: D = [1.000, 1.000]
  Критерий 2: D = [0.000, 0.143]

=== Шаг 3. Интервальные метрики S_i и R_i ===
A1: S = [0.549, 0.500], R = [0.500, 0.500]
A2: S = [0.363, 0.347], R = [0.227, 0.266]
A3: S = [0.500, 0.572], R = [0.500, 0.500]

=== Шаг 4. Интервальные экстремумы ===
S*: [0.363, 0.347]
S-: [0.549, 0.572]
R*: [0.227, 0.266]
R-: [0.500, 0.500]

=== Шаг 5. Интервальная мера компромисса Q_i ===
A1: Q = [1.000, 0.828]
A2: Q = [0.000, 0.032]
A3: Q = [0.878, 1.000]

=== Шаг 6. Центры интервалов и ранжирование ===
A2: Q_center = 0.016
A1: Q_center = 0.914
A3: Q_center = 0.939

==